![caratula](img/caratula.png)

# Índice
1. Introducción
2. Modelo Conceptual
3. Modelos Acoplados
4. Experimentación
5. Conclusiones

# 1. Introducción

Es frecuente encontrar en las calles lo que en el contexto de este trabajo llamamos "zonas comerciales", es decir, zonas en donde se encuentran muchos comercios cerca. Esto en general genera comportamientos particulares, como por ejemplo:

* **La concentración de personas en estas zonas**, debido a que allí esperan encontrar mas variedad y opciones a la hora de adquirir un nuevo bien o servicio.
* **Aumento en la cantidad de ventas en los locales de estas zonas**, por la mayor actividad de personas en la zona.
* **Aumento de impuestos en los locales de estas zonas**, porque la zona adquiere *importancia*.

En muchos casos los comercios generan mayores ganancias al estar en estas zonas comerciales pero en muchos otros las ganancias no bastan para cubrir los costos de alquiler (entre los que se encuentran el aumento de impuestos). Además, si se generan estas zonas comerciales, los comercios que quedan fuera pueden llegar a verse afectados por sus ganancias por pérdida de clientes.

En este trabajo proponemos simular una economía básica para ver como fluctuan los comercios, ver si se forman zonas comerciales (teniendo en cuenta las preferencias de la gente de dirigirse a zonas con más locales) y, en caso de surgir, analizar cómo es que las zonas comerciales impactan sobre las ganancias de los locales dentro y fuera de estas zonas.


# 2. Modelo Conceptual

Este modelo cuenta con dos agentes: el local y la persona. La idea del modelo es que las personas vayan moviéndose libremente por el mapa. En cada paso que den pueden tomar la decisión de gastar plata en un local o de usar sus ahorros para abrir uno nuevo. Todas las personas cobran periodicamente un salario que les permite hacer lo mencionado.

Para los locales queremos que estos queden abiertos solamente si generan suficientes ganancias. Para eso cada local tiene que pagar periódicamente un impuesto y, de no alcanzar a pagarlo, el local se cierra.

## Simplificaciones del modelo

En esta sección dejamos anotadas las restricciones con las que armamos el modelo:

* Todas las personas tienen empleo y ganan el mismo sueldo.
* Los locales una vez abiertos no están relacionados con la persona que los abrió, se vuelven entes independientes.
* Todos los locales ofrecen el mismo producto.
* Los costos de los productos son iguales (c) y los impuestos a pagar también.

## Reglas de las personas

* Se mueven de manera aleatoria pero con mayor predilección por celdas con locales.
* No pueden compartir posición
* Luego de un período $k$ de tiempo reciben un sueldo $s$
* Si una persona tiene mas de $m$ plata, con cierta probabilidad $p_{open}$ puede abrir un local en donde esta parada.
* Cuando se encuentra parado en un local elije con probabilidad $p_{buy}$ consumir allí el producto de precio $c$ (solo si tiene la plata suficiente).

## Reglas de los locales

* Periódicamente deben pagar pagar un costo $i$ para mantenerse abiertos (impuestos).
* Si no puede pagar el impuesto se cierra.

# 3. Modelos Acoplados

Además del modelo *top.ma*, que es el que utilizamos para la simulación, contamos con el modelo acoplado Cell-Devs *persona*, que será detallado a continuación.

Para empezar, este modelo va a utilizar 4 capas:
* **Capa 0 (Dinero personas)**: En cada celda contiene el dinero de la persona (si es que la hay) o 0 en caso contrario.
* **Capa 1 (Próxima dirección de personas)**: En cada celda contiene un valor $d \in \{1,2,3,4\}$, cuyos valores representan una de las 4 direcciones posibles que tomará la persona como próximo movimiento (en caso de que pueda ser efectuado).
* **Capa 2 (Switch salario/impuestos)**: Cada cierto tiempo, va a escribir un valor no nulo en todas sus celdas (inicializadas en 0), que va a ser que las personas (capa 0) cobren su salario y los locales (capa 3) paguen sus impuestos.
* **Capa 3 (Dinero locales)**: En cada celda contiene el dinero del local (si es que hay) o 0 en caso contrario.

Por otro lado, definiremos las direcciones y el vecindario como muestra la siguiente figura:

<img src="img/direcciones+vecindario.jpg" style="width: 500px;" />

Dicho esto, podemos empezar a definir las reglas.


### Reglas de movimiento
Queremos modelar personas moviéndose libremente por el mapa; esto es, dada una persona, ésta debería mirar en la capa 1 (direcciones) ĺa dirección a donde debería moverse y ver si efectivamente puede realizar el movimiento. El movimiento podrá realizarse si:
* No hay otra persona en la posición donde quiere ir.
* No hay otra persona con mayor prioridad que quiera moverse a la misma posición donde quiere ir. En caso de conflicto, vamos a decidir arbitrariamente que la persona de la derecha va a tener prioridad para moverse.

La lógica de movimiento van a estar definidas por las reglas que corresponden a la persona "llegando a la celda actual" y las que corresponden a la persona "saliendo de la celda actual". Para las primeras, la siguientes reglas describen el comportamiento esperado:

<img src="img/reglas-persona-viniendo.jpg" style="width: 400px;"/>

```
% ------- MOVIMIENTO: PERSONA CONTIGUA SE MUEVE A CELDA ACTUAL ------- 
rule : {(0,-1,0)} 100 { cellpos(2) = 0 and (0,0,0) = 0 and (0,-1,0) > 0 and (0,-1,1) = 3 } 
rule : {(-1,0,0)} 100 { cellpos(2) = 0 and (0,0,0) = 0 and (-1,0,0) > 0 and (-1,0,1) = 4 and ((0,-1,0) = 0 or (0,-1,1) != 3)}
rule : {(0,1,0)}  100 { cellpos(2) = 0 and (0,0,0) = 0 and (0,1,0)  > 0 and (0,1,1) = 1  and ((0,-1,0) = 0 or (0,-1,1) != 3) and ((-1,0,0) = 0 or (-1,0,1) != 4)}
rule : {(1,0,0)}  100 { cellpos(2) = 0 and (0,0,0) = 0 and (1,0,0)  > 0 and (1,0,1) = 2  and ((0,-1,0) = 0 or (0,-1,1) != 3) and ((-1,0,0) = 0 or (-1,0,1) != 4) and ((0,1,0) = 0 or (0,1,1) != 1)}
```

En cuanto a las reglas que corresponden a la persona "saliendo de la celda actual",

<img src="img/reglas-persona-saliendo.jpg" style="width: 400px;"/>

```
% ------- MOVIMIENTO: PERSONA EN CELDA ACTUAL SE MUEVE A CELDA CONTIGUA ------- 
rule : 0 100 {cellpos(2) = 0 and (0,0,0) != 0 and (0,1,0)  = 0 and (0,0,1) = 3 }
rule : 0 100 {cellpos(2) = 0 and (0,0,0) != 0 and (1,0,0)  = 0 and (0,0,1) = 4 and ((1,-1,0)  = 0 or (1,-1,1) != 3)}
rule : 0 100 {cellpos(2) = 0 and (0,0,0) != 0 and (0,-1,0) = 0 and (0,0,1) = 1 and ((0,-2,0)  = 0 or (0,-2,1) != 3) and ((-1,-1,0) = 0 or (-1,-1,1) != 4)} 
rule : 0 100 {cellpos(2) = 0 and (0,0,0) != 0 and (-1,0,0) = 0 and (0,0,1) = 2 and ((-1,-1,0) = 0 or (-1,-1,1) != 3) and ((-2,0,0) = 0 or (-2,0,1) != 4) and ((-1,1,0) = 0 or (-1,1,1) != 1)}
```

### Reglas de apertura de locales

Queremos que, con cierta probabilidad $p_{open}$, una persona con el suficiente dinero ($>m$) pueda abrir un local en la capa 3. Notar que para que la persona abra un local, tienen que pasar 2 cosas a la vez:
* La persona pierde $m$ de su dinero (inversión en abrir local, capa 0)
* Un local gana $m$ de dinero (inversión inicial local, capa 3)

Para esto, vamos a considerar un estado intermedio (conceptual) en donde si se abre un local, éste va a abrirse con una inversión incial de -1 de forma tal que (en tiempo 0):
* La persona al ver que hay un local "sin inversión inicial" pueda descontar el monto de inversión de su dinero.
* El local al ver que no tiene "inversion inicial" pueda tomarla.

La siguiente figura describe el comportamiento esperado:

<img src="img/reglas-abrir-local.jpg" style="width: 600px;"/>

```
% ------- ABRIR LOCAL ------- 
rule : {-1} 0 { cellpos(2) = 3 and (0,0,1) > {{ m }} and (0,0,0) = 0 and random < {{ p_open }}}
rule : {(0,0,0) - {{ m }}} 0 { cellpos(2) = 0 and (0,0,3) = -1}
rule : { {{ m }} } 0 { cellpos(2) = 3 and (0,0,0) = -1}
```

### Reglas de cobro de salarios y pago de impuestos
Para hacer que las personas cobren su sueldo y los locales paguen sus impuestos cada cierto tiempo, vamos a utilizar la capa 2 como medio. Cuando pase cierto tiempo, vamos a poner todas las celdas de esta capa en 2 (momento en el cual los locales tienen que pagar), luego lo pasamos a 1 (momento en el cual las personas tienen que cobrar), y luego a 0 (default), utilizando las siguientes reglas:

```
% ------- SWITCH SALARIO/IMPUESTOS ------- 
rule : 2 1000 {cellpos(2) = 2 and (0,0,0) = 0}
rule : 1 0 {cellpos(2) = 2 and (0,0,0) = 2}
rule : 0 0 {cellpos(2) = 2 and (0,0,0) = 1}
```

Cuando una celdas de esta capa esta en 2, entonces el local (si es que hay) de la capa 3 tendrá que pagar sus impuestos. Ahora bien, si un local no puede pagar sus impuestos (porque no tiene fondos suficientes) entonces cierra.

```
% ------- LOCALES PAGAN IMPUESTOS CUANDO SWITCH SE ACTIVA------- 
rule : {(0,0,0) - {{ i }}} 0 { cellpos(2) = 3 and (0,0,0) >= {{ i }} and (0,0,3) = 1}
rule : {0} 0 { cellpos(2) = 3 and (0,0,0) < {{ i }} and (0,0,3) = 1}
```

Por otro lado, cuando las celdas de la capa 2 pasen a valor 1, vamos a hacer que todas las personas cobren su sueldo.

```
% ------- PERSONAS COBRAN SALARIO CUANDO SWITCH SE ACTIVA ------- 
rule : {(0,0,0) + {{ s }}} 0 { cellpos(2) = 0 and (0,0,0) != 0 and (0,0,2) = 2}
```

### Reglas de consumo de productos
Cuando una persona compra un producto en un local, ocurren 2 cosas al mismo tiempo:
* La persona pierde $p_{buy}$ de dinero.
* El local gana $p_{buy}$ de dinero.

Para generar ambos eventos al mismo tiempo, vamos a generar un estado intermedio (conceptual) análogo a como hicimos antes para evitarnos tener que crear otra capa. En particular, vamos a hacer que cuando una persona compre en un local, se descuente de su sueldo y además cambie su sueldo de signo, de forma tal que:
* Si el local ve que la persona tiene sueldo negativo, entonces le vendió el producto, por lo cual procede a sumarse $p_buy$
* Si la persona ve que su sueldo es negativo, entonces compró un producto en el local, por lo cual simplemente lo invierte para volver al estado normal.

Las reglas de consumo de la persona son muy parecidas a las de movimiento, a diferencia de que ahora se agrega que, con cierta probabilidad, la persona que "viene a la celda actual", consume en el local por lo cual se mueve con su sueldo actualizado después de realizar la compra (y con el signo opuesto):
```
% ------- MOVIMIENTO: PERSONA CONTIGUA SE MUEVE A CELDA ACTUAL (CONSUMIENDO SI HAY LOCAL) ------- 
rule : {-1 * ((0,-1,0) - {{ c }})} 100 { cellpos(2) = 0 and (0,0,0) = 0 and (0,-1,0) > {{ c }} and (0,-1,1) = 3 and (0,0,3) > 0 and random < {{ p_buy }}} 
rule : {-1 * ((-1,0,0) - {{ c }})} 100 { cellpos(2) = 0 and (0,0,0) = 0 and (-1,0,0) > {{ c }} and (-1,0,1) = 4 and ((0,-1,0) = 0 or (0,-1,1) != 3) and (0,0,3) > 0 and random < {{ p_buy }}}
rule : {-1 * ((0,1,0) - {{ c }})}  100 { cellpos(2) = 0 and (0,0,0) = 0 and (0,1,0)  > {{ c }} and (0,1,1) = 1  and ((0,-1,0) = 0 or (0,-1,1) != 3) and ((-1,0,0) = 0 or (-1,0,1) != 4) and (0,0,3) > 0 and random < {{ p_buy }}}
rule : {-1 * ((1,0,0) - {{ c }})}  100 { cellpos(2) = 0 and (0,0,0) = 0 and (1,0,0) > {{ c }} and (1,0,1) = 2  and ((0,-1,0) = 0 or (0,-1,1) != 3) and ((-1,0,0) = 0 or (-1,0,1) != 4) and ((0,1,0) = 0 or (0,1,1) != 1) and (0,0,3) > 0 and random < {{ p_buy }}}
```

Notar que, si no hay local en la celda actual o la persona decide no comprar, estas reglas no se evalúan y entonces se evalúan las reglas originales de movimiento (sin consumo). Pero además, ahora hay que agregar la regla de que si no puede moverse y está parado sobre un local, entonces puede llegar a volver a consumir.

```
% ------- SI ME QUEDE EN EL LUGAR, CONSUMO ---------
rule : {-1 * ((0,0,0) - {{ c }})}  100 { cellpos(2) = 0 and (0,0,0) > {{ c }} and random < {{ p_buy }}}
```

Luego, si la persona consumió en un local (y entonces su dinero quedó invertido), hay que hacer que el local gane el dinero que corresponde y la persona vuelva a invertir su sueldo para que quede de forma apropiada, en tiempo 0.

```
% ------- LOCAL GANA PLATA CUANDO PERSONA CONSUME ------- 
rule: {(0,0,0) + {{ c }}} 0 { cellpos(2) = 3 and (0,0,0) > 0 and (0,0,1) < 0}
rule: {-1 * (0,0,0)} 0 { cellpos(2) = 0 and (0,0,0) < 0} 
```

### Reglas de generación de próxima dirección
Para generar la próxima dirección, queremos movernos con mayor probabilidad para algún lado en donde haya más locales. Para esto, vamos a considerar las siguientes reglas:
```
rule : {1} 100 {cellpos(2) = 1 and random <= ({{ cant_locales_izquierda }} / 4)}
rule : {2} 100 {cellpos(2) = 1 and random <= ({{ cant_locales_arriba }} / 4)}
rule : {3} 100 {cellpos(2) = 1 and random <= ({{ cant_locales_derecha }} / 4)}
rule : {4} 100 {cellpos(2) = 1 and random <= ({{ cant_locales_abajo }} / 4)}
rule : {randInt(3) + 1} 100 {cellpos(2) = 1}
```

Con las siguientes macros definidas:
```
"cant_locales_arriba": "(if((-2,0,2) > 0, 1, 0) + if((-1,0,2) > 0, 1, 0) + if((-1,-1,2) > 0, 1, 0) + if((-1,1,2) > 0, 1, 0))"
"cant_locales_abajo": "(if((1,0,2) > 0, 1, 0) + if((2,0,2) > 0, 1, 0) + if((1,1,2) > 0, 1, 0) + if((1,-1,2) > 0, 1, 0))"
"cant_locales_derecha": "(if((0,1,2) > 0, 1, 0) + if((0,2,2) > 0, 1, 0) + if((-1,1,2) > 0, 1, 0) + if((1,1,2) > 0, 1, 0))"
"cant_locales_izquierda": "(if((0,-1,2) > 0, 1, 0) + if((0,-2,2) > 0, 1, 0) + if((-1,-1,2) > 0, 1, 0) + if((1,-1,2) > 0, 1, 0))"
```

De esta forma, la probabilidad de moverse a cada uno de los lados esta ponderado por la cantidad de locales que tiene. 

### Modelo entero

```
[top]
components : persona

[persona]
type : cell
border : wrapped
delay : inertial
defaultDelayTime : 100
dim : (10, 10, 4)
initialCellsValue : top.val
initialValue : 0
localTransition : persona-rule


neighbors :                                  persona(-2,0,3)
neighbors :                 persona(-1,-1,3) persona(-1,0,3) persona(-1,1,3)
neighbors : persona(0,-2,3) persona(0,-1,3)  persona(0,0,3)  persona(0,1,3) persona(0,2,3)
neighbors :                 persona(1,-1,3)  persona(1,0,3)  persona(1,1,3)
neighbors :                                  persona(2,0,3)

neighbors :                                  persona(-2,0,2)
neighbors :                 persona(-1,-1,2) persona(-1,0,2) persona(-1,1,2)
neighbors : persona(0,-2,2) persona(0,-1,2)  persona(0,0,2)  persona(0,1,2) persona(0,2,2)
neighbors :                 persona(1,-1,2)  persona(1,0,2)  persona(1,1,2)
neighbors :                                  persona(2,0,2)

neighbors :                                  persona(-2,0,1)
neighbors :                 persona(-1,-1,1) persona(-1,0,1) persona(-1,1,1)
neighbors : persona(0,-2,1) persona(0,-1,1)  persona(0,0,1)  persona(0,1,1) persona(0,2,1)
neighbors :                 persona(1,-1,1)  persona(1,0,1)  persona(1,1,1)
neighbors :                                  persona(2,0,1)

neighbors :                                  persona(-2,0,0)
neighbors :                 persona(-1,-1,0) persona(-1,0,0) persona(-1,1,0)
neighbors : persona(0,-2,0) persona(0,-1,0)  persona(0,0,0)  persona(0,1,0) persona(0,2,0)
neighbors :                 persona(1,-1,0)  persona(1,0,0)  persona(1,1,0)
neighbors :                                  persona(2,0,0)

[persona-rule]
% ------- LOCAL GANA PLATA CUANDO PERSONA CONSUME ------- 
rule: {(0,0,0) + {{ c }}} 0 { cellpos(2) = 3 and (0,0,0) > 0 and (0,0,1) < 0}
rule: {-1 * (0,0,0)} 0 { cellpos(2) = 0 and (0,0,0) < 0}

% ------- PERSONAS COBRAN SALARIO CUANDO SWITCH SE ACTIVA ------- 
rule : {(0,0,0) + {{ s }}} 0 { cellpos(2) = 0 and (0,0,0) != 0 and (0,0,2) = 2}

% ------- LOCALES PAGAN IMPUESTOS CUANDO SWITCH SE ACTIVA ------- 
rule : {(0,0,0) - {{ i }}} 0 { cellpos(2) = 3 and (0,0,0) >= {{ i }} and (0,0,3) = 1}
rule : {0} 0 { cellpos(2) = 3 and (0,0,0) < {{ i }} and (0,0,3) = 1}

% ------- GENERACION PROX DIRECCION ------- 
rule : {1} 100 {cellpos(2) = 1 and random <= ({{ cant_locales_izquierda }} / 4) }
rule : {2} 100 {cellpos(2) = 1 and random <= ({{ cant_locales_arriba }} / 4) }
rule : {3} 100 {cellpos(2) = 1 and random <= ({{ cant_locales_derecha }} / 4) }
rule : {4} 100 {cellpos(2) = 1 and random <= ({{ cant_locales_abajo }} / 4) }
rule : {randInt(3) + 1} 100 {cellpos(2) = 1}

% ------- ABRIR LOCAL ------- 
rule : {-1} 0 { cellpos(2) = 3 and (0,0,1) > {{ m }} and (0,0,0) = 0 and random < {{ p_open }}}
rule : {(0,0,0) - {{ m }}} 0 { cellpos(2) = 0 and (0,0,3) = -1}
rule : { {{ m }} } 0 { cellpos(2) = 3 and (0,0,0) = -1}

% ------- MOVIMIENTO: PERSONA CONTIGUA SE MUEVE A CELDA ACTUAL (CONSUMIENDO SI HAY LOCAL) ------- 
rule : {-1 * ((0,-1,0) - {{ c }})} 100 { cellpos(2) = 0 and (0,0,0) = 0 and (0,-1,0) > {{ c }} and (0,-1,1) = 3 and (0,0,3) > 0 and random < {{ p_buy }}} 
rule : {-1 * ((-1,0,0) - {{ c }})} 100 { cellpos(2) = 0 and (0,0,0) = 0 and (-1,0,0) > {{ c }} and (-1,0,1) = 4 and ((0,-1,0) = 0 or (0,-1,1) != 3) and (0,0,3) > 0 and random < {{ p_buy }}}
rule : {-1 * ((0,1,0) - {{ c }})}  100 { cellpos(2) = 0 and (0,0,0) = 0 and (0,1,0)  > {{ c }} and (0,1,1) = 1  and ((0,-1,0) = 0 or (0,-1,1) != 3) and ((-1,0,0) = 0 or (-1,0,1) != 4) and (0,0,3) > 0 and random < {{ p_buy }}}
rule : {-1 * ((1,0,0) - {{ c }})}  100 { cellpos(2) = 0 and (0,0,0) = 0 and (1,0,0) > {{ c }} and (1,0,1) = 2  and ((0,-1,0) = 0 or (0,-1,1) != 3) and ((-1,0,0) = 0 or (-1,0,1) != 4) and ((0,1,0) = 0 or (0,1,1) != 1) and (0,0,3) > 0 and random < {{ p_buy }}}

% ------- MOVIMIENTO: PERSONA CONTIGUA SE MUEVE A CELDA ACTUAL (SIN CONSUMIR)------- 
rule : {(0,-1,0)} 100 { cellpos(2) = 0 and (0,0,0) = 0 and (0,-1,0) > 0 and (0,-1,1) = 3 } 
rule : {(-1,0,0)} 100 { cellpos(2) = 0 and (0,0,0) = 0 and (-1,0,0) > 0 and (-1,0,1) = 4 and ((0,-1,0) = 0 or (0,-1,1) != 3)}
rule : {(0,1,0)}  100 { cellpos(2) = 0 and (0,0,0) = 0 and (0,1,0)  > 0 and (0,1,1) = 1  and ((0,-1,0) = 0 or (0,-1,1) != 3) and ((-1,0,0) = 0 or (-1,0,1) != 4)}
rule : {(1,0,0)}  100 { cellpos(2) = 0 and (0,0,0) = 0 and (1,0,0)  > 0 and (1,0,1) = 2  and ((0,-1,0) = 0 or (0,-1,1) != 3) and ((-1,0,0) = 0 or (-1,0,1) != 4) and ((0,1,0) = 0 or (0,1,1) != 1)}

% ------- MOVIMIENTO: PERSONA EN CELDA ACTUAL SE MUEVE A CELDA CONTIGUA ------- 
rule : 0 100 {cellpos(2) = 0 and (0,0,0) != 0 and (0,1,0)  = 0 and (0,0,1) = 3 }
rule : 0 100 {cellpos(2) = 0 and (0,0,0) != 0 and (1,0,0)  = 0 and (0,0,1) = 4 and ((1,-1,0)  = 0 or (1,-1,1) != 3)}
rule : 0 100 {cellpos(2) = 0 and (0,0,0) != 0 and (0,-1,0) = 0 and (0,0,1) = 1 and ((0,-2,0)  = 0 or (0,-2,1) != 3) and ((-1,-1,0) = 0 or (-1,-1,1) != 4)} 
rule : 0 100 {cellpos(2) = 0 and (0,0,0) != 0 and (-1,0,0) = 0 and (0,0,1) = 2 and ((-1,-1,0) = 0 or (-1,-1,1) != 3) and ((-2,0,0) = 0 or (-2,0,1) != 4) and ((-1,1,0) = 0 or (-1,1,1) != 1)}

% ------- SI ME QUEDE EN EL LUGAR, CONSUMO ---------
rule : {-1 * ((0,0,0) - {{ c }})}  100 { cellpos(2) = 0 and (0,0,0) > {{ c }} and random < {{ p_buy }}}

% ------- SWITCH SALARIO/IMPUESTOS ------- 
rule : 2 1000 {cellpos(2) = 2 and (0,0,0) = 0}
rule : 1 0 {cellpos(2) = 2 and (0,0,0) = 2}
rule : 0 0 {cellpos(2) = 2 and (0,0,0) = 1}

% ------- DEFAULT ------- 
rule : {(0,0,0)} 100 {t}
```

# 4. Experimentación

Todas las simulaciones pueden reproducirse en la notebook *simulaciones.ipynb* ubicadas en este mismo directorio. Notar que, salvo que se especifique lo contrario, se consideraron los siguientes parámetros fijos:
```
    "salario_personas": 10,
    "impuestos_local": 2,
    "costo_poner_local": 15,
    "proba_poner_local": 0.25,
    "proba_consumir": 0.9,
    "costo_consumicion": 1
```

## 4.1 Variando impuestos de locales

Lo primero que queremos ver es como afecta en la economía aumentar los impuestos de los locales. Para ello, vamos a correr una simulación por 2 días considerando los parámetros fijos especificados anteriormente y variando el monto de impuestos a pagar por los locales. Partimos del siguiente estado inicial:

<img src="img/exp-imp-inicial.jpg" style="width: 600px;"/>

Los resultados se muestran a continuación:
<img src="img/exp-imp.jpg" style="width: 800px;"/>

Lo primero interesante a observar es la tendencia en gral a formarse grupos de locales. Esto viene dado por la preferencia de las personas a moverse por zonas comerciales. También vemos incluso que las zonas comerciales más grandes suelen generar mayores ingresos.

En relación a la variación de impuestos, podemos ver que cuando los impuestos son *2* o *4* la economía prospera y se forman zonas comerciales, con locales con que se mantienen en el tiempo debido a las ganancias generadas. Entre estos casos, vemos que, como es de esperar, cuanto menores son los impuestos a pagar, más ganancias se generan. 

Con los parámetros considerados, se puede ver que cuando los impuestos son de *6* o más, la economía se vuelve inestable y los l


# 5. Conclusiones